In [14]:
! pip install accelerate -U

In [1]:
!nvidia-smi

Sun Dec 10 01:10:02 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 536.67                 Driver Version: 536.67       CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                     TCC/WDDM  | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 4070      WDDM  | 00000000:24:00.0  On |                  N/A |
|  0%   34C    P8               9W / 200W |    912MiB / 12282MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
import pickle
from sklearn.model_selection import train_test_split

# Data

In [3]:
lyrics = pickle.load(open('french_lyrics_only_v2.pickle', 'rb'))

In [4]:
song_lyrics = ['\n'.join(lines) for artist, songs in lyrics.items() for title, lines in songs.items()]

In [5]:
print(song_lyrics[1])

Et nous, plébéiens d'à-présent
Anciens Egyptiens, nouveaux roturiers bohémiens
En aspirant à la justice, nous n’aspirâmes qu'au statut d'humain

Et nous demain, vilains d’aujourd'hui, manants d'avant-hier
Fils honni par ses propres pères
En aspirant à l'égalité, nous n'aspirâmes qu'au statut d'humain

Et nous, prolétaires en bas de l'échelle alimentaire
Animal servile se rêvant prince de la ville
En aspirant à la fraternité, nous n’aspirâmes qu’au statut d'humain
C’est comme si ma peau finissait jaunie à cause de mon gilet noir
Le malheur ne discrimine point, te voilà maintenant parlant de moi
Ô ciel, quelle méprisable condition qu'est devenue la mienne
Je ne peux plus dire à qui que ce soit que j'aime
C'est comme si ma peau finissait jaunie à cause de mon gilet noir
Le malheur ne discrimine point, te voilà maintenant parlant de moi
C’est la vie, c'est la France, c'est le monde
C'est la fronde, la galère qui nous poussent à l'extrême
C'est l'enfance qu'on regrette, plus personne dit je

In [6]:
train, test = train_test_split(song_lyrics,test_size=0.15)

In [9]:
def build_text_files(data_text, dest_path):
    f = open(dest_path, 'w', encoding="utf-8")
    data = ''
    for text in data_text:
        data += text + ' '
    f.write(data)

In [10]:
build_text_files(train,'train_dataset.txt')
build_text_files(test,'test_dataset.txt')

print("Train dataset length: "+str(len(train)))
print("Test dataset length: "+ str(len(test)))

Train dataset length: 13485
Test dataset length: 2380


In [11]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("asi/gpt-fr-cased-small")

train_path = 'train_dataset.txt'
test_path = 'test_dataset.txt'

In [12]:
from transformers import TextDataset, DataCollatorForLanguageModeling

def load_dataset(train_path,test_path,tokenizer):
    train_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=train_path,
          block_size=128)
     
    test_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=test_path,
          block_size=128)   
    
    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, mlm=False,
    )
    return train_dataset,test_dataset,data_collator

train_dataset,test_dataset,data_collator = load_dataset(train_path,test_path,tokenizer)

C:\Users\harri\anaconda3\Lib\site-packages\transformers\data\datasets\language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


# Model

In [13]:
from transformers import Trainer, TrainingArguments,AutoModelWithLMHead

model = AutoModelWithLMHead.from_pretrained("asi/gpt-fr-cased-small")

C:\Users\harri\anaconda3\Lib\site-packages\transformers\models\auto\modeling_auto.py:1468: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(


In [15]:
import accelerate

In [16]:
training_args = TrainingArguments(
    output_dir="./gpt-fr-rap", #The output directory
    overwrite_output_dir=True, #overwrite the content of the output directory
    num_train_epochs=2, # number of training epochs
    per_device_train_batch_size=16, # batch size for training
    per_device_eval_batch_size=16,  # batch size for evaluation
    eval_steps = 400, # Number of update steps between two evaluations.
    save_steps=800, # after # steps model is saved 
    warmup_steps=500,# number of warmup steps for learning rate scheduler
    prediction_loss_only=True,
    )

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

In [17]:
trainer.train()

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ········


wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\harri/.netrc


Step,Training Loss
500,4.022700
1000,3.324400
1500,3.224000
2000,3.179600
2500,3.123800
3000,3.096200
3500,3.072100
4000,3.057800
4500,3.025800
5000,3.013900


TrainOutput(global_step=11162, training_loss=3.0422457907067857, metrics={'train_runtime': 2045.9963, 'train_samples_per_second': 87.286, 'train_steps_per_second': 5.456, 'total_flos': 1.1665774706688e+16, 'train_loss': 3.0422457907067857, 'epoch': 2.0})

In [18]:
trainer.save_model()

# Inference

In [23]:
from transformers import pipeline

rapper = pipeline('text-generation', model='./gpt-fr-rap', tokenizer='asi/gpt-fr-cased-small', )
                #config={'max_length':200, 'output_scores':True})

In [44]:
start = 'Chez moi'
res = rapper(start, max_new_tokens=100, do_sample=True, temperature=0.9, top_k=10, top_p=0.92, num_return_sequences=3)

In [45]:
print(res[0]['generated_text'])

Chez moi, y a pas de place pour la haine, j'ai la haine, j'ai la haine
Y a la vie, le monde, le monde, le monde, le monde, le monde, le monde
[Kaaris]
La vie c'est une course poursuite, la vie c'est une poursuite
La vie c'est une course poursuite, la vie c'est une poursuite
La vie c'est une course poursuite, la vie c'est une


In [46]:
print(res[1]['generated_text'])

Chez moi, t'es qu'une sale pute, tu m'as donne des ailes
Tu sais qu'on a tous les memes problemes, on a tous les memes problemes
T'inquiete, j'fais tout pour la famille, tu sais qu'on a tous les memes problemes
T'inquiete, j'fais tout pour la famille, tu sais qu'on a tous les memes problemes
On a tous les memes problemes


In [47]:
print(res[2]['generated_text'])

Chez moi y'a pas de place pour l'erreur
T'as vu la vie en rose, c'est pas un film
Mais tu m'as vu en rose, tu m'as vu en rose
T'as vu la vie en rose, c'est pas un film
Mais tu m'as vu en rose, tu m'as vu en rose
Tu m'as vu en rose, tu m'as vu en rose
T'as vu la vie en


In [53]:
start = "De mon retour des pays des merveilles, "
res = rapper(start, max_new_tokens=100, do_sample=True, temperature=0.9, top_k=10, top_p=0.92, num_return_sequences=3)

In [54]:
print(res[0]['generated_text'])

De mon retour des pays des merveilles, 
j'suis un enfant du peuple, j'ai appris a faire le sourd
j'suis un enfant du peuple, j'ai appris a faire le sourd
j'suis un enfant du peuple, j'ai appris a faire le sourd
j'suis un enfant du peuple, j'ai appris a faire le sourd
j'suis un enfant du peuple, j'ai appris a faire le sourd
j'suis un enfant du peuple, j'ai appris a faire


In [55]:
print(res[1]['generated_text'])

De mon retour des pays des merveilles, 
Je suis un jeune, j'ai des choses a raconter, 
J'ai pas envie de faire du rap, je veux juste faire du rap 
Et je suis un jeune, j'ai des choses a raconter, 
J'ai pas envie de faire du rap, je veux juste faire du rap 
Et je suis un jeune, j'ai des choses a raconter, 
Je suis un jeune, j'ai des choses a raconter, 
Et


In [56]:
print(res[2]['generated_text'])

De mon retour des pays des merveilles, 
Je suis un jeune d'ici, et moi, je ne vois que toi 
Tu as le sourire des enfants mais t'es trop belle 
T'es trop mignonne, t'es trop jolie 
Je ne t'ai pas dit que je te connaissais 
Tu ne veux pas me connaitre 
T'es trop jolie, t'es trop jolie 
T'es trop mignonne, t'es trop jolie 
Je t'ai dit que je


In [57]:
start = "Baby veut du salot"
res = rapper(start, max_new_tokens=100, do_sample=True, temperature=0.9, top_k=10, top_p=0.92, num_return_sequences=3)

In [58]:
print(res[0]['generated_text'])

Baby veut du salot mais elle est pas prete
Mais elle est pas prete
Et je sais qu'elle est pas prete
Elle veut du salo, j'la fais monter sur l'terrain
J'la fais monter sur l'terrain, j'la fais monter sur l'terrain
J'la fais monter sur l'terrain, j'la fais monter sur l'terrain
J'la fais monter sur l'terrain, j'la


In [59]:
print(res[1]['generated_text'])

Baby veut du salot, j'lui dis : "T'es pas comme nous"
J'la prends dans l'froc et j'me casse, elle m'en veut, j'lui dis : "T'es pas comme nous"
Elle m'dit : "T'es pas comme nous"
Et j'suis dans l'froc, j'lui dis : "T'es pas comme nous"
J'lui dis : "T'es pas comme nous


In [60]:
print(res[2]['generated_text'])

Baby veut du salot mais j'me sens mal a la tete
C'est pour mes freres qui m'ont donne la vie, c'est pour mes freres qui m'ont donne la vie
Et si j'te dis c'est que c'est vrai, c'est vrai
C'est vrai, c'est vrai
C'est vrai, c'est vrai
C'est vrai, c'est vrai
C'est vrai, c'est vrai
